# Tutorial: Causal Inference 03 - Uplift Tree and Interpretability

Audience:
- Students interested in interpretable treatment targeting.

Prerequisites:
- Notebooks 01 and 02.

Learning goals:
- Train a KL-based uplift tree.
- Read and discuss tree structure.
- Connect segment-level patterns to policy decisions.


## Outline

1. Fit uplift tree.
2. Inspect tree summary text.
3. Validate uplift ranking by score bins.
4. Exercise + pitfall + extension.


In [ ]:
from pathlib import Path
import sys

project_root = Path.cwd().resolve()
if not (project_root / "src").exists():
    project_root = project_root.parent

sys.path.insert(0, str(project_root / "src"))

import numpy as np
import pandas as pd

from causal_showcase.data import load_marketing_ab_data, train_test_split_prepared
from causal_showcase.modeling import fit_uplift_tree

data_path = project_root / "data" / "raw" / "marketing_ab.csv"
prepared = load_marketing_ab_data(data_path)
train_data, test_data = train_test_split_prepared(prepared)

tree_result = fit_uplift_tree(train_data, test_data)
print("Uplift tree fitted.")


## Step 1 - Read the first lines of the learned tree

This summary is useful in class to discuss how the model segments users.


In [ ]:
lines = tree_result.tree_summary.splitlines()
print("\n".join(lines[:35]))


## Step 2 - Empirical uplift by predicted-uplift bins

If the tree ranking works, high-score bins should show higher observed uplift.


In [ ]:
scores = tree_result.uplift_scores
bins = pd.qcut(scores, q=5, duplicates="drop")

df_eval = pd.DataFrame({"bin": bins, "y": test_data.outcome, "w": test_data.treatment})
rows = []
for bin_label, g in df_eval.groupby("bin", observed=False):
    treated = g.loc[g["w"] == 1, "y"]
    control = g.loc[g["w"] == 0, "y"]
    uplift = float(treated.mean() - control.mean()) if len(treated) and len(control) else float("nan")
    rows.append({"uplift_bin": str(bin_label), "n": len(g), "empirical_uplift": uplift})

pd.DataFrame(rows).sort_values("empirical_uplift", ascending=False)


## Exercises, pitfalls, and extension

- Exercise: Reduce `max_depth` and compare ranking strength.
- Pitfall: Overfitting with deep trees and tiny leaves.
- Extension: Compare uplift tree against best meta-learner from Notebook 2.


In [ ]:
# Answer scaffold: modify this dictionary and re-train.
experiment_settings = {
    "max_depth": 3,
    "min_samples_leaf": 400,
    "min_samples_treatment": 200,
}
experiment_settings
